## 處理流程 ETL
- 下載檔案 Extract
- 基本的檔案處理 ( 解壓縮, 轉換格式...等) Transformation
- 資料格式處理 T
- 資料輸出 Loading

## 下載檔案

In [1]:
import requests
def download_file(folder_path,url):
    local_filename = folder_path+url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename


## 檔案處理

In [11]:
import zipfile
def upzip(zip_path,unzip_path):
    print(zip_path)
    zf = zipfile.ZipFile(zip_path,'r',zipfile.ZIP_DEFLATED)#打开已存在的zip文件
    zf.extractall(unzip_path)#解压到当前目录

## 格式處理

In [23]:
import xlrd
def extract_bank(row_s,d):
    bank = {"銀行類別":"本國銀行","時間":d,"業務項目":"進口開狀"}
    name_zh = sh.cell_value(rowx=row_s,colx = 0).replace("\u3000","")
    this_month = int(sh.cell_value(rowx=row_s,colx = 2).replace(",",""))*1000
    sum_month = int(sh.cell_value(rowx=row_s,colx = 5).replace(",",""))*1000
    bank["本月"] = this_month
    bank["本年累計"] = sum_month
    bank["銀行中文名稱"] = name_zh
    return bank


## 寫出資料 Loading

In [24]:
def writeToLC(date,banks):
    with open('./import_LC_'+date+'.csv','w+') as f:
        f.write("銀行中文名稱,本月,本年累計,銀行類別,時間,業務項目")
        for bank in banks:
            f.write("\n")
            f.write(bank["銀行中文名稱"]+",")
            f.write("%d,"%(bank["本月"]))
            f.write("%d,"%(bank["本年累計"]))
            f.write(bank["銀行類別"]+",")
            f.write(bank["時間"]+",")
            f.write(bank["業務項目"])

## 完整流程

In [25]:
def month_job(date):
    filepath = "12081400(%d)"%date
    download_file("./","http://www.banking.gov.tw/webdowndoc?file=/stat/lc/%s.zip"%filepath)
    upzip(filepath+".zip","./")
    book = xlrd.open_workbook("%s.xls"%filepath)
    sh = book.sheet_by_index(1)
    banks = []
    read_Flag = 0
    index = 0

    while 1:
        ## 狀態控制
        if read_Flag == 0 and sh.cell_value(rowx=index,colx = 0).replace("\u3000","")=="總計":
            read_Flag = 1
        if read_Flag == 1 and sh.cell_value(rowx=index,colx = 0).replace("\u3000","") =="":
            break
        ## 資料處理
        if read_Flag == 1:
            banks.append(extract_bank(index,'%d'%(date+191100)))
        index = index + 1
    writeToLC('%d'%(date+191100),banks) 

In [26]:
month_job(10511)

12081400(10511).zip


NameError: name 'sh' is not defined